In [111]:
import sys
os.environ['METALHOME'] = '/dfs/scratch1/saelig/slicing/metal/'
import metal
import os
# Import other dependencies
import torch
import torch.nn as nn
import torch.nn.functional as F

# Set random seed for notebook
SEED = 123

In [75]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load Data

Here, the train/test split was defined in the dataset. We then split the train set into a train/valid (see next cell)

In [157]:
from skimage import io, transform
import torchvision.transforms as transforms
import numpy as np
DATASET_DIR = '/lfs/1/saelig/CUB_200_2011/'
IMAGES_DIR = os.path.join(DATASET_DIR, 'images')

#Size of eac
image_list = np.loadtxt(os.path.join(DATASET_DIR, 'images.txt'), dtype=str)
train_test_split = np.loadtxt(os.path.join(DATASET_DIR, 'train_test_split.txt'), dtype=int)
labels = np.loadtxt(os.path.join(DATASET_DIR, 'image_class_labels.txt'), dtype=int)

X = []
Y = []
X_test = []
Y_test = []

#image size (332, 500, 3)
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
tt = transforms.ToTensor()

for image_id, image_file in image_list:
    image_id = int(image_id)
    image_data = io.imread(os.path.join(IMAGES_DIR, image_file))
    image_data = transform.resize(image_data, (64,64,3)) #resize all images to 32x32
    label = labels[image_id - 1][1]
    if train_test_split[image_id - 1][1] == 1: #put in train
        X.append(image_data)
        Y.append(label)
    else: #put in test
        X_test.append(image_data)
        Y_test.append(label)

X_train = np.stack(X)
Y_train = np.array(Y)
X_test = np.stack(X_test)
Y_test = np.array(Y_test)

Now let's convert all the data to tensors, and create a validation set.

In [158]:
from metal.utils import split_data
X_train = np.stack(X)
Y_train = np.array(Y)

(X_train, X_valid), (Y_train, Y_valid) = split_data(X_train, Y_train, splits=[0.8,0.2], seed=SEED)
X_test = np.stack(X_test)
Y_test = np.array(Y_test)

X_train, X_valid, X_test = torch.tensor(X_train), torch.tensor(X_valid), torch.tensor(X_test)
Y_train, Y_valid, Y_test = torch.tensor(Y_train), torch.tensor(Y_valid), torch.tensor(Y_test)

X_train = X_train.permute(0,3,1,2)
X_valid = X_valid.permute(0,3,1,2)
X_test = X_test.permute(0,3,1,2)

Create a task. Use a resnet50 for now as our input model. Since the resnet already includes the fully connected layer, we don't specify a `head_module`, which defaults to the identity.

In [164]:
#from torchvision.models.resnet import *
from metal.mmtl.slicing.tasks import MultiClassificationTask
from metal.mmtl.metal_model import MetalModel 
from resnet import *

resnet_model = resnet50(num_classes=200)

task0 = MultiClassificationTask(
    name='BirdClassificationTaask', 
    input_module=resnet_model, 
)
tasks = [task0]
model = MetalModel(tasks, verbose=False)

Create payload abstraction for our train/valid/test sets.

In [183]:
from metal.mmtl.payload import Payload
from pprint import pprint

payloads = []
splits = ["train", "valid", "test"]
X_splits = X_train, X_valid, X_test
Y_splits = Y_train, Y_valid, Y_test

for i in range(3):
    payload_name = f"Payload{i}_{splits[i]}"
    task_name = task0.name
    print(X_splits[i].shape)
    payload = Payload.from_tensors(payload_name, {'data': X_splits[i]}, {'labels' : Y_splits[i]}, task_name, splits[i], batch_size=32)
    #payload = Payload.from_tensors(payload_name, X_splits[i], Y_splits[i], task_name, splits[i], batch_size=32)
    payloads.append(payload)

pprint(payloads)

torch.Size([4795, 3, 64, 64])
torch.Size([1199, 3, 64, 64])
torch.Size([5794, 3, 64, 64])
[Payload(Payload0_train: labels_to_tasks=[{'labels': 'BirdClassificationTaask'}], split=train),
 Payload(Payload1_valid: labels_to_tasks=[{'labels': 'BirdClassificationTaask'}], split=valid),
 Payload(Payload2_test: labels_to_tasks=[{'labels': 'BirdClassificationTaask'}], split=test)]


In [184]:
from metal.mmtl.trainer import MultitaskTrainer

trainer = MultitaskTrainer()

In [186]:
scores = trainer.train_model(
    model, 
    payloads, 
    n_epochs=20, 
    log_every=2,
    lr=0.02, 
    progress_bar=False,
)

Beginning train loop.
Expecting a total of approximately 4800 examples and 150 batches per epoch from 1 payload(s) in the train split.
batch:  torch.Size([32, 12288])
torch.Size([32, 12288])


Exception ignored in: <generator object tqdm_notebook.__iter__ at 0x7f4c9c4f6a40>
Traceback (most recent call last):
  File "/dfs/scratch0/saelig/miniconda3/envs/slicing/lib/python3.6/site-packages/tqdm/_tqdm_notebook.py", line 226, in __iter__
    self.sp(bar_style='danger')
AttributeError: 'tqdm_notebook' object has no attribute 'sp'


RuntimeError: Expected 4-dimensional input for 4-dimensional weight [64, 3, 7, 7], but got 2-dimensional input of size [32, 12288] instead